In [2]:
from pathlib import Path

import appose
from appose.service import TaskStatus

In [2]:
import sys 
print(4,file=sys.stderr)
print(4)
#3

4


4


In [4]:
def check_task(task):
    if task.status != TaskStatus.COMPLETE:
        print(">>> Task FAILED")
    else:
        print(">>> Task OK, value =", task.outputs)

In [4]:
env = appose.system()
with env.python() as s:    
    task = s.task("import sys; print(3,file=sys.stderr)",inputs={'a':1})
    task.wait_for()
    check_task(task)
print(task)

>>> Task OK, value = {}
self.uuid='64391a203e8f4d16bf35b86c19e3f756', self.status=<TaskStatus.COMPLETE: 'COMPLETE'>, self.error=None


In [ ]:
#task on worker side
# print(task) <appose.python_worker.Task object at 0x00000242517D7A10>
# task on python server side
# task self.uuid='6322f5f2e8de4e87aef0b34a24ae7ee8', self.status=<TaskStatus.COMPLETE: 'COMPLETE'>, self.error=None
# print(task) <appose.service.Task at 0x190517a0590>

In [4]:
import appose
env = appose.system()
with env.python() as sv:    
    task = sv.task("task.update('hello!'); task.outputs['msg']='ok'; print('boo')") 
    task.listen(lambda e: print("WORKER: " , e.message, type(e.message))  )
    task.wait_for()
    print(task.status)
    print(task.outputs)

WORKER:  None <class 'NoneType'>
WORKER:  hello! <class 'str'>
WORKER:  None <class 'NoneType'>
TaskStatus.COMPLETE
{'msg': 'ok'}


In [5]:

script = """
task.outputs['a']=5;
task.outputs['b']=8
"""

env = appose.system()
with env.python() as service1:
    task = service1.task(script)    
    task.listen(lambda e: print("WORKER: " , e.message))   
    task.wait_for()    
    #check_task(task)
    print(task.status)
    print(task.outputs)


WORKER:  None
WORKER:  None
TaskStatus.COMPLETE
{'a': 5, 'b': 8}


In [1]:
import appose
env = appose.system()
with env.python() as svc:
    #svc.debug(print)
    task = svc.task("task.update('hello!'); task.outputs['msg']='ok'")
    task.listen(lambda e: print("WORKER:", e.message))
    task.wait_for()
    print(task.status)
    print(task.outputs)

WORKER: None
WORKER: hello!
WORKER: None
TaskStatus.COMPLETE
{'msg': 'ok'}


In [7]:
import appose
from appose.service import TaskStatus

env = appose.system()

with env.python() as service:
    #service.debug(print)  # so you see SERVICE-0 logs

    script = """task.outputs["value"] = 42; v = 100"""

    task = service.task(script)
    task.wait_for()

    print("status :", task.status)
    print("error  :", task.error)
    print("outputs:", task.outputs)

    if task.status != TaskStatus.COMPLETE:
        print(">>> Task FAILED")
    else:
        print(">>> Task OK, value =", task.outputs["value"])
        

print(task)
task.outputs,task

status : TaskStatus.COMPLETE
error  : None
outputs: {'value': 42}
>>> Task OK, value = 42
self.uuid='e266d8af65b44e9ba2d241032197703c', self.status=<TaskStatus.COMPLETE: 'COMPLETE'>, self.error=None


({'value': 42}, <appose.service.Task at 0x132e558d6e0>)

In [8]:
task.script

'task.outputs["value"] = 42; v = 100'

In [17]:
[k for k in dir(appose.types) if not k.startswith('_')]

['Any',
 'Args',
 'Dict',
 'NDArray',
 'Sequence',
 'SharedMemory',
 'Union',
 'ceil',
 'decode',
 'encode',
 'json',
 'prod',
 're',
 'resource_tracker',
 'shared_memory']

In [18]:
narr = appose.NDArray(dtype='float32', shape=[2,3,2])
narrx = narr.ndarray()
type(narr),narr, narr.shm, narr.shape, type(narrx), narrx

(appose.types.NDArray,
 SharedMemory('wnsm_60e898ce', size=48),
 [2, 3, 2],
 numpy.ndarray,
 array([[[0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.]]], dtype=float32))

In [3]:
import appose
narr = appose.NDArray( dtype ='uint8',shape=[2,3,2])
try: narrx = narr.ndarray()
except Exception as e: print(e)
#narrx[:,:,:]=4
#narrx[0,0,0]=100
#narr.ndarray()

In [2]:
import appose
from appose.service import TaskStatus

env = appose.system()

script = """
task.update('starting')
task.update(f'{narr}')
task.update(f'{type(narr)}')
#narrx = narr.ndarray()
task.update(f'{narr.shape}')
task.update(f'{narr.shm}')
#narr.ndarray()
#try: narrx = narr.ndarray()
#except Exception as e: task.update(e)
#narrx[:,:,:]=4
#narrx[0,0,0]=100
#task.outputs['value'] = narrx.sum()
task.update('finished assignment')
""" 

narr = appose.NDArray( dtype ='uint8',shape=[2,3,2])

with env.python() as service:
    #service.debug(print)
    task = service.task(script,inputs={'narr':narr})
    task.listen(lambda e: print("WORKER: " , e.message))
    task.wait_for()    

#task.error, task.outputs, task.status, task.inputs #, task.script
print(narr.ndarray())

WORKER:  None
WORKER:  starting
WORKER:  NDArray(dtype='uint8', shape=[2, 3, 2], shm='wnsm_3c58603a' (12))
WORKER:  <class 'appose.types.NDArray'>
WORKER:  [2, 3, 2]
WORKER:  SharedMemory('wnsm_3c58603a', size=4096)
WORKER:  finished assignment
WORKER:  None
[[[0 0]
  [0 0]
  [0 0]]

 [[0 0]
  [0 0]
  [0 0]]]


In [8]:
a= [2,3]
id(a)

1288378509120

In [11]:
import appose
from appose.service import TaskStatus

env = appose.system()

script = """
task.update('starting')
task.update(f'{narr}')
task.update(f'{type(narr)}')
narr.append(4)
task.outputs['value'] = sum(narr)
task.update(f'{narr}')
task.update(f'{id(narr)}')
task.update('finished assignment')
""" 

narr = [1,2,3] #appose.NDArray( dtype ='uint8',shape=[2,3,2])
print(id(narr))
with env.python() as service:
    #service.debug(print)
    task = service.task(script,inputs={'narr':narr})
    task.listen(lambda e: print("WORKER: " , e.message))
    task.wait_for()    
print(id(narr))

print(task.error, task.outputs, task.status, task.inputs) #, task.script
print(narr)

1288381604224
WORKER:  None
WORKER:  starting
WORKER:  [1, 2, 3]
WORKER:  <class 'list'>
WORKER:  [1, 2, 3, 4]
WORKER:  1821789872384
WORKER:  finished assignment
WORKER:  None
1288381604224
None {'value': 10} TaskStatus.COMPLETE {'narr': [1, 2, 3]}
[1, 2, 3]


In [ ]:
import appose
from appose.service import TaskStatus

env = appose.system()

script = """
import traceback
task.update('1. starting')
task.update(f'2. {narr.shape}')
try:
    task.update(f'3. {narr}')
    task.update(f'4. {type(narr)}')

    # this is the line that causes trouble
    arr = narr.ndarray()
    task.update(f'5. ndarray: shape={arr.shape}, dtype={arr.dtype}')

    task.update('finished assignment')
    task.outputs["ok"] = True

except Exception as e:
    task.outputs["error"] = traceback.format_exc()
    raise
    
""" 

with env.python() as service:
    service.debug(print)
    with appose.NDArray(dtype='float32', shape=[2,3,4]) as narr:
        task = service.task(script,inputs={'narr':narr})
        task.listen(lambda e: print("WORKER: " , e.message, "\n----\n"))
        task.wait_for()    

print("STATUS:", task.status)
print("TASK ERROR:", task.error)
print("TASK OUTPUT error:\n", task.outputs.get("error"))
print("driver-side ndarray:\n", narr.ndarray())

[SERVICE-8] <worker stderr closed>
[SERVICE-8] {"task":"8fbfc3f98cda4084a36106f157efc018","requestType":"EXECUTE","script":"\nimport traceback\ntask.update('1. starting')\ntask.update(f'2. {narr.shape}')\ntry:\n    task.update(f'3. {narr}')\n    task.update(f'4. {type(narr)}')\n\n    # this is the line that causes trouble\n    arr = narr.ndarray()\n    task.update(f'5. ndarray: shape={arr.shape}, dtype={arr.dtype}')\n\n    task.update('finished assignment')\n    task.outputs[\"ok\"] = True\n\nexcept Exception as e:\n    task.outputs[\"error\"] = traceback.format_exc()\n    raise\n\n","inputs":{"narr":{"appose_type":"ndarray","dtype":"float32","shape":[2,3,4],"shm":{"appose_type":"shm","name":"wnsm_b223a0cd","rsize":96}}},"queue":null}
[SERVICE-8] {"task":"8fbfc3f98cda4084a36106f157efc018","responseType":"LAUNCH"}

WORKER:  None 
----

[SERVICE-8] {"message":"1. starting","info":null,"task":"8fbfc3f98cda4084a36106f157efc018","responseType":"UPDATE"}

WORKER:  1. starting 
----

[SERVICE

In [ ]:
script ="""
#import cellpose
#import numpy
import numpy as np
task.update("loaded cellpose")
#task.outputs["cellpose_version"] = cellpose.version
"""

import appose
from appose.service import TaskStatus
env = appose.system()
with env.python() as service:
    #service.debug(print)
    task = service.task(script)
    task.listen(lambda e: print("WORKER: " , e.message))
    task.wait_for()    
print(task.error, task.outputs, task.status, task.inputs) 

WORKER:  None


In [1]:
import sys
sys.executable

'C:\\Users\\varghesechac\\.local\\share\\appose\\cellpose3-uv\\Scripts\\python.exe'

In [1]:
%%time
import cellpose
cellpose.version

CPU times: total: 1.86 s
Wall time: 1.87 s


'3.0'